In [ ]:
### empty directory
! rm -rf ./ssd/data/*

# How to have a project in action do not perform this action

In [ ]:
# set your categories
CATEGORIES = ['car', 'mouse', 'remote']

# Set project directory
data = "custom"

In [ ]:
import os

Annotations = "./ssd/data/" + data + "/Annotations/"
ImageSets = "./ssd/data/" + data + "/ImageSets/Main/"
ImageSetsTrain = ImageSets + "train.txt"
ImageSetsTrainval = ImageSets + "trainval.txt"
ImageSetsTest = ImageSets + "test.txt"
ImageSetsVal = ImageSets + "val.txt"
JPEGImages = "./ssd/data/" + data + "/JPEGImages/"
labels = "./ssd/data/" + data + "/labels.txt"

# Check and create directory
directories = [Annotations, ImageSets, JPEGImages]

for directory in directories:
    if not os.path.exists(directory):
        os.makedirs(directory)

# The above are the steps that must be performed. The first of the following two is to save the picture, and the second is to display the saved picture and delete it.

In [ ]:
# Check device number
!ls -ltrh /dev/video*

In [ ]:
from jetcam.usb_camera import USBCamera
from jetcam.csi_camera import CSICamera

# for USB Camera (Logitech C270 webcam), uncomment the following line
camera = USBCamera(width=640, height=480, capture_device=0) # confirm the capture_device number

# for CSI Camera (Raspberry Pi Camera Module V2), uncomment the following line
# camera = CSICamera(width=224, height=224, capture_device=0) # confirm the capture_device number

camera.running = True
print("camera created")

In [ ]:
import cv2
import datetime
import random
import ipywidgets
import traitlets
import ipywidgets as widgets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget

points = [{
    "cat": CATEGORIES[0],
    "top": True,
    "topX": -1,
    "topY": -1,
    "bottomX": -1,
    "bottomY": -1,
    "color": (random.randint(1, 255), random.randint(1, 255), random.randint(1, 255))
}]
point = 0


# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

# create image preview
camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)


# snapshot_widget = ipywidgets.Image(value=default_image, width=camera.width, height=camera.height)
# traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)
def darm(camera):
    global points, point
    snapshot = camera.copy()
    for i, p in enumerate(points):
        # print(p)
        if p['bottomX'] >= 0 & p['bottomY'] >= 0:
            # 创建一个和原图像同样大小的透明图像
            overlay = snapshot.copy()

            # 在透明图像上绘制矩形
            if point == i:
                cv2.rectangle(overlay, (p['topX'], p['topY']), (p['bottomX'], p['bottomY']), p['color'] + (255,), 4)
                alpha = 1  # 设置透明度
            else:
                cv2.rectangle(overlay, (p['topX'], p['topY']), (p['bottomX'], p['bottomY']), p['color'] + (100,), 4)
                alpha = 0.2  # 设置透明度

            # 使用alpha blending将透明图像和原图像混合在一起
            
            snapshot = cv2.addWeighted(overlay, alpha, snapshot, 1 - alpha, 0)
        else:
            snapshot = cv2.circle(snapshot, (p['topX'], p['topY']), 4, p['color'], 3)
    return bgr8_to_jpeg(snapshot)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=darm)



# update counts when we select a new category

def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        if points[point]['top']:
            points[point]['top'] = False
            points[point]['topX'] = data['offsetX']
            points[point]['topY'] = data['offsetY']
            points[point]['top_x_widget'].value = points[point]['topX']
            points[point]['top_y_widget'].value = points[point]['topY']
            points[point]['bottomX'] = -1
            points[point]['bottomY'] = -1
            points[point]['bottom_x_widget'].value = points[point]['bottomX']
            points[point]['bottom_y_widget'].value = points[point]['bottomY']
            
        else:
            points[point]['top'] = True
            points[point]['bottomX'] = data['offsetX']
            points[point]['bottomY'] = data['offsetY']
            points[point]['bottom_x_widget'].value = points[point]['bottomX']
            points[point]['bottom_y_widget'].value = points[point]['bottomY']
            
        

        
camera_widget.on_msg(save_snapshot)


points_widget = ipywidgets.VBox([])

def add_ponit(p, index):
    # create widgets
    category_widget = ipywidgets.Dropdown(options=CATEGORIES, description='category')

    top_x_widget = ipywidgets.IntText(description='Top X:')
    top_y_widget = ipywidgets.IntText(description='Top Y:')
    bottom_x_widget = ipywidgets.IntText(description='Bottom X:')
    bottom_y_widget = ipywidgets.IntText(description='Bottom Y:')

    def update_top_x_widget(change):
        # global topX
        p['topX'] = change['new']
    top_x_widget.observe(update_top_x_widget, names='value')

    def update_top_y_widget(change):
        # global topY
        p['topY'] = change['new']
    top_y_widget.observe(update_top_y_widget, names='value')

    def update_bottom_x_widget(change):
        # global bottomX
        p['bottomX'] = change['new']
    bottom_x_widget.observe(update_bottom_x_widget, names='value')

    def update_bottom_y_widget(change):
        # global bottomY
        p['bottomY'] = change['new']
    bottom_y_widget.observe(update_bottom_y_widget, names='value')
    
    choose_button = ipywidgets.Button(description='Choose')

    
    del_button = ipywidgets.Button(description='Del')

    
    box_layout = widgets.Layout(border='3px solid rgb{}'.format(p['color'][::-1]), margin_bottom='10px', background_color='lightblue')
    point_widget = ipywidgets.VBox([
        ipywidgets.HBox([category_widget, choose_button, del_button]),
        ipywidgets.HBox([top_x_widget, top_y_widget,]),
        ipywidgets.HBox([bottom_x_widget, bottom_y_widget]),
    ], layout=box_layout)
    p['category_widget'] = category_widget
    p['top_x_widget'] = top_x_widget
    p['top_y_widget'] = top_y_widget
    p['bottom_x_widget'] = bottom_x_widget
    p['bottom_y_widget'] = bottom_y_widget
    p['choose_button'] = choose_button
    p['point_widget'] = point_widget
    
    def choose(button):
        global point
        point = index
        for xi, xp in enumerate(points):
            if point == xi:
                choose_button.style.button_color = '#4898F8'
            else:
                xp['choose_button'].style.button_color = None
        
    choose_button.on_click(choose)
    
    def del_b(button):
        global point
        new_children = [child for child in points_widget.children if child != point_widget]
        points_widget.children = tuple(new_children)
        if len(points) - 1 == index:
            point = point - 1
        del points[index]

        points[point]['choose_button'].style.button_color = '#4898F8'

    del_button.on_click(del_b)
    points_widget.children = (*points_widget.children, point_widget)
    return p

for index, p in enumerate(points):
    add_ponit(p, index)
    if point == index:
        p['choose_button'].style.button_color = '#4898F8'
    else:
        p['choose_button'].style.button_color = None


# Add this to create a button for saving images
save_button = ipywidgets.Button(description='Save Image')

# Add this to define a function that will be called when the button is clicked
def save_image(b):
    global data, camera, points
    
    # 获取当前时间
    current_time = datetime.datetime.now().strftime("%Y%m%d")

    # 生成10位随机数
    random_number = random.randint(1000000000, 9999999999)

    # 构建文件名
    filename = f"{current_time}_{random_number}"
    
    cv2.imwrite(JPEGImages + filename+'.jpg', camera.value)
    
    # Save the bounding box coordinates and category to a text file
    with open(Annotations + filename + '.xml', 'a') as txtfile:  # Change 'data.txt' to your desired filename
        # Write the data for this image
        txtfile.write(f"""<annotation>
    <filename>{filename}</filename>
    <folder>{data}</folder>
    <source>
        <database>{data}</database>
        <annotation>{data}</annotation>
        <image>{data}</image>
    </source>
    <size>
        <width>{camera.width}</width>
        <height>{camera.height}</height>
        <depth>{camera.value.shape[2]}</depth>
    </size>
    <segmented>0</segmented>
""")
        for p in points:
            txtfile.write(f"""    <object>
        <name>{p['category_widget'].value}</name>
        <pose>unspecified</pose>
        <truncated>0</truncated>
        <difficult>0</difficult>
        <bndbox>
            <xmin>{p['bottomX'] if p['topX'] >= p['bottomX'] else p['topX']}</xmin>
            <ymin>{p['bottomY'] if p['topY'] >= p['bottomY'] else p['topY']}</ymin>
            <xmax>{p['bottomX'] if p['topX'] <= p['bottomX'] else p['topX']}</xmax>
            <ymax>{p['bottomY'] if p['topY'] <= p['bottomY'] else p['topY']}</ymax>
        </bndbox>
    </object>
""")
        txtfile.write(f"""</annotation>""")  # Separate entries with a blank line
        # 打开文件以追加写入模式
    with open(ImageSetsTrain, "a") as file:
        # 写入新的文本行
        file.write(filename+"\n")
    
    with open(ImageSetsTrainval, "a") as file:
        # 写入新的文本行
        file.write(filename+"\n")

    with open(ImageSetsTest, "a") as file:
        # 写入新的文本行
        file.write(filename+"\n")
        
    with open(ImageSetsVal, "a") as file:
        # 写入新的文本行
        file.write(filename+"\n")
        
    with open(labels, 'w') as file:
        for category in CATEGORIES:
            file.write(category + '\n')
    
    print('Image saved.')

# Add this to make the button call the function when clicked
save_button.on_click(save_image)



def add_point_new(button):
    
    global add_point, ponits
    
    p = add_ponit({
    "cat": CATEGORIES[0],
    "top": True,
    "topX": -1,
    "topY": -1,
    "bottomX": -1,
    "bottomY": -1,
    "color": (random.randint(1, 255), random.randint(1, 255), random.randint(1, 255))
}, len(points))
    points.append(p)
    
    
    
add_point_button = ipywidgets.Button(description='Add New Point')
add_point_button.on_click(add_point_new)



data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_widget, ]),
    add_point_button,
    points_widget,
    save_button
])

display(data_collection_widget)
print("data_collection_widget created")

# Show all training images of the project you have saved

## It takes time to load the list, and the delete button can only be clicked after the loading is complete. The delete button is invalid if the loading is not completed

1. When the background of the button turns blue, it means it can be clicked
1. The button disappears after clicking, indicating that the deletion is complete

In [ ]:
import os
import cv2
import xml.etree.ElementTree as ET
import ipywidgets
import ipywidgets as widgets
from ipywidgets import GridBox
from IPython.display import display
from PIL import Image
import random

# Get list of all files in the directory
images_files = os.listdir(JPEGImages)
xmls_files = os.listdir(Annotations)

# Separate the image and XML files
image_files = [file for file in images_files if file.endswith('.jpg')]
xml_files = [file for file in xmls_files if file.endswith('.xml')]

# Sort files based on creation time in descending order
image_files.sort(key=lambda x: os.path.getctime(os.path.join(JPEGImages, x)), reverse=True)
xml_files.sort(key=lambda x: os.path.getctime(os.path.join(Annotations, x)), reverse=True)

colors = {}
# 设置颜色
for c in CATEGORIES:
    colors[c] = (random.randint(1, 200), random.randint(1, 200), random.randint(1, 200))
    color_widget = widgets.ColorPicker(
        concise=False,
        description=c,
        value='#{:02x}{:02x}{:02x}'.format(*colors[c][::-1]),
        disabled=True
    )
    display(color_widget)

# 创建一个空的GridBox
grid = GridBox([], layout=ipywidgets.Layout(grid_template_columns="repeat(auto-fill, 155px)"))

# 显示GridBox
display(grid)

def del_content(filename, jpg):
    print(filename, jpg)
    # 提取文件名中的基本部分（不包含扩展名）
    base_filename = os.path.splitext(jpg)[0]

    # 读取文件
    with open(filename, 'r') as file:
        lines = file.readlines()

    # 删除与指定文件名相同的行
    lines = [line for line in lines if not line.startswith(base_filename)]

    # 保存修改后的文件
    with open(filename, 'w') as file:
        file.writelines(lines)

def on_delete_button_clicked(b, image_file, xml_file, show, showNew):
    # Delete the image and XML file
    os.remove(os.path.join(JPEGImages, image_file))
    os.remove(os.path.join(Annotations, xml_file))

    del_content(ImageSetsTrain, image_file)
    del_content(ImageSetsTrainval, image_file)
    del_content(ImageSetsTest, image_file)
    del_content(ImageSetsVal, image_file)

    # Remove the image and XML file from the lists
    image_files.remove(image_file)
    xml_files.remove(xml_file)

    # Remove the associated grid element from the GridBox widget
    # new_children = [child for child in grid.children if child[0].children[1].value != image_file]
    show.children = tuple(showNew)

    print("removed")
        
# Loop over each image/XML pair
for index, (image_file, xml_file) in enumerate(zip(image_files, xml_files)):
    # Parse the XML file
    tree = ET.parse(os.path.join(Annotations, xml_file))
    root = tree.getroot()

    # Open the image file
    im = cv2.imread(os.path.join(JPEGImages, image_file))

    # Loop over each 'object' element in the XML file
    for obj in root.iter('object'):
        # Get the bounding box coordinates
        bndbox = obj.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)

        # Draw the bounding box on the image
        background_color = colors[obj.find('name').text]
        cv2.rectangle(im, (xmin, ymin), (xmax, ymax), background_color, 10)
        
        # 添加文本
        text = obj.find('name').text
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 2
        font_thickness = 2
        text_size, _ = cv2.getTextSize(text, font, font_scale, font_thickness)

        # 计算背景框的位置和大小
        text_width = text_size[0]
        text_height = text_size[1]
        background_width = text_width + 10
        background_height = text_height + 10

        background_origin = (xmin, ymin)
        background_bottom_right = (background_origin[0] + background_width, background_origin[1] + background_height)

        # 绘制背景框
        cv2.rectangle(im, background_origin, background_bottom_right, background_color, -1)

        # 添加文本
        text_origin = (background_origin[0] + 5, background_origin[1] + text_height + 5)
        text_color = (255, 255, 255)
        cv2.putText(im, text, text_origin, font, font_scale, text_color, font_thickness)

    # Convert the image to JPEG format
    _, jpeg_image = cv2.imencode('.jpg', im)

    # Create an Image widget
    image_widget = widgets.Image(
        value=jpeg_image.tobytes(),
        format='jpg',
        width=148,
    )
    
    # Create a button to delete the current image and XML file
    delete_button = widgets.Button(description="Delete")
    delete_button.disabled = True

    # Create a label widget to display the file name
    file_label = widgets.Label(value=image_file)
    
    show = ipywidgets.VBox([file_label, image_widget, delete_button], layout=ipywidgets.Layout(border='1px solid #ccc'))
    
    # Set the delete button's click event to the function
    delete_button.on_click(lambda b, img=image_file, xml=xml_file, show=show, showNew=[file_label, image_widget]: on_delete_button_clicked(b, img, xml, show, showNew))
    
    # vbox.children = (*vbox.children, show)
    grid.children = (*grid.children, show)

for g in grid.children:
    # print(g.children)
    g.children[2].disabled = False
    g.children[2].style.button_color = '#4898F8'

# use docker desktop client to training

1. Download the latest docker client from the docker.com website
2. Double-click the client to install, and restart after the installation is complete
3. If you are using it for the first time and the network environment is not good, you can download the tf.tar image in advance and import it into docker. The pytorch.tar image address: [pytorch.tar](https://netorg639112-my.sharepoint.com/:u:/g/personal/jack_wang_integem_com1/EYLyfiGFA9pDolh--WUt_DYBqE8C203INvq99Fj0I4nGbA?e=ifglj8), [the specific process](#import-image)
4. Open cmd on the desktop and run
> Copy and paste all the content below into the terminal and press enter
```bash
docker run --name nvidia --rm -it -p 8888:8888 integem/pytorch:latest bash -c "cd /home && wget https://raw.githubusercontent.com/integemjack/tf-online/main/ssd.ipynb && jupyter lab --allow-root"
```

5. Get the token address, for example:

```bash
# http://127.0.0.1:8888/lab?token=21941d36957a9aa8e7401f930d3cbf7fe8232733b4ebcb5d
# Hold down ctrl and click on the link to open jupyter lab
```

## ------------------------------------------------------------------------

# export image

Open cmd and run the following command to generate a pytorch.tar image file in the set directory

```bash
docker save -o <pytorch.tar absolute path> integem/pytorch:latest
```

# import image

You can download the pytorch.tar file through [pytorch.tar](https://netorg639112-my.sharepoint.com/:u:/g/personal/jack_wang_integem_com1/EYLyfiGFA9pDolh--WUt_DYBqE8C203INvq99Fj0I4nGbA?e=ifglj8), or you can get pytorch.tar through the above export

```bash
docker load -i <pytorch.tar absolute path>
```

# Run test

In [ ]:
! pip3 install paho-mqtt

In [ ]:
# If you need to convert the model into an onnx file in nano, please execute the following
%cd ssd
! python3 onnx_export.py --model-dir=models/custom

In [ ]:
# Start the mqtt service

import requests
import os

# MQTT server address
os.environ["MQTT_IP"] = "localhost"
os.environ["MQTT_PORT"] = "1883"

os.environ["PC_IP"] = "192.168.20.71"

url = f'http://localhost/connect?machineid=car&mqtt_ip={os.environ["MQTT_IP"]}:{os.environ["MQTT_PORT"]}'
headers = {'Cache-Control': 'no-cache'}
response = requests.get(url, headers=headers)

# 确保请求成功
if response.status_code == 200:
    print(f"{response.text}")
else:
    print('Error:', response.status_code)

In [ ]:
import paho.mqtt.client as mqtt

# Define a callback function when a connection is established with the server
def on_connect(client, userdata, flags, rc):
    if rc == 0:
        print("Connected successfully.")
    else:
        print("Connection failed with error code %d." % rc)

# Create an MQTT client object
client = mqtt.Client()

# Set connection callback function
client.on_connect = on_connect

# Connect to the MQTT server
client.connect(os.environ["MQTT_IP"], int(os.environ["MQTT_PORT"]))

# Start handling all messages and communications
client.loop_start()


In [ ]:
# Start the detectnet program, it may take a while to run for the first time
client.publish("car", f'start|detectnet|input=v4l2:///dev/video0|output=rtp://{os.environ["PC_IP"]}:1234|width=640|height=480|model_path=/home/nvidia/jetson-inference/python/training/detection/ssd/models/{data}/ssd-mobilenet.onnx|class_labels_path=/home/nvidia/jetson-inference/python/training/detection/ssd/models/{data}/labels.txt|output_bbox=boxes|input_blob=input_0|output_cvg=scores')

In [ ]:
# Close the detectnet program
client.publish("car", "stop")